### SageMaker Counterfactual prediction

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)
step_func_role = "arn:aws:iam::{}:role/AmazonSageMaker-StepFunctionsWorkflowExecutionRole".format(account_id)

In [2]:

version_tag="202309120312"
pytorch_custom_image_name=f"large-scale-ptm-ppi:gpu-{version_tag}"
instance_type = "ml.p3.2xlarge" #"ml.g4dn.2xlarge"  #ml.g4dn.2xlarge
instance_count = 1



In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

In [4]:
bucket = "aegovan-data"

In [5]:

dataset = { 
    
    "contrastimpo": "s3://aegovan-data/contrast-set/test_imdb_contrast_plus_original.tsv",
    "contrastimdb": "s3://aegovan-data/contrast-set/test_imdb_contrast.tsv",
    "yelp" :  "s3://aegovan-data/yelp_polarity/test.csv",
    "semeval": "s3://aegovan-data/semeval2017taskb/SemEval2017-task4-test.subtask-BD.english.txt",
    "amazon":  "s3://aegovan-data/amazon_reviews_polarity/test.csv",



}

dataset = { 
    
    "contrastimpo": "s3://aegovan-data/contrast-set/test_imdb_contrast_plus_original.tsv",
    "contrastimdb": "s3://aegovan-data/contrast-set/test_imdb_contrast.tsv"



}

dataset_factory_map = {"contrastimpo": "contrastimdb"}


In [6]:
import datetime
date_fmt = datetime.datetime.today().strftime("%Y%m%d%H%M")

In [7]:
imdb_bert_2k_training_jobs = [
"imdb202304231701-89-00-00-05",
"imdb202304231701-28-00-00-05",
"imdb202304231701-18-00-00-05",
"imdb202304231701-09-00-00-05",
"imdb202304231701-00-00-00-05",
"imdb202304231701-88-00-00-04",
"imdb202304231701-28-00-00-04",
"imdb202304231701-18-00-00-04",
"imdb202304231701-09-00-00-04",
"imdb202304231701-00-00-00-04",
"imdb202304231701-89-00-00-03",
"imdb202304231701-28-00-00-03",
"imdb202304231701-18-00-00-03",
"imdb202304231701-09-00-00-03",
"imdb202304231701-00-00-00-03",
"imdb202304231701-88-00-00-02",
"imdb202304231701-28-00-00-02",
"imdb202304231701-18-00-00-02",
"imdb202304231701-09-00-00-02",
"imdb202304231701-00-00-00-02",
"imdb202304231701-88-00-00-01",
"imdb202304231701-28-00-00-01",
"imdb202304231701-19-00-00-01",
"imdb202304231701-09-00-00-01",
"imdb202304231701-00-00-00-01"
]

In [8]:

training_jobs_bert_imdb_5h = [
"imdb202306041410-88-00-00-05",
"imdb202306041410-26-00-00-05",
"imdb202306041410-18-00-00-05",
"imdb202306041410-09-01-00-05",
"imdb202306041410-00-00-00-05",
"imdb202306041410-92-00-00-04",
"imdb202306041410-28-00-00-04",
"imdb202306041410-18-00-00-04",
"imdb202306041410-09-00-00-04",
"imdb202306041410-00-00-00-04",
"imdb202306041410-87-00-00-03",
"imdb202306041410-28-00-00-03",
"imdb202306041410-20-00-00-03",
"imdb202306041410-09-00-00-03",
"imdb202306041410-00-00-00-03",
"imdb202306041410-90-00-00-02",
"imdb202306041410-27-00-00-02",
"imdb202306041410-18-00-00-02",
"imdb202306041410-08-00-00-02",
"imdb202306041410-00-00-00-02",
"imdb202306041410-88-00-00-01",
"imdb202306041410-29-00-00-01",
"imdb202306041410-17-00-00-01",
"imdb202306041410-09-00-00-01",
"imdb202306041410-00-00-00-01"
]

In [9]:


training_jobs_roberta_imdb_5h = [
    "imdb-rob-5h-202309101303-88-00-00-05",
"imdb-rob-5h-202309101303-26-00-00-05",
"imdb-rob-5h-202309101303-18-00-00-05",
"imdb-rob-5h-202309101303-09-01-00-05",
"imdb-rob-5h-202309101303-00-00-00-05",
"imdb-rob-5h-202309101303-92-00-00-04",
"imdb-rob-5h-202309101303-28-00-00-04",
"imdb-rob-5h-202309101303-18-00-00-04",
"imdb-rob-5h-202309101303-09-00-00-04",
"imdb-rob-5h-202309101303-00-00-00-04",
"imdb-rob-5h-202309101303-87-00-00-03",
"imdb-rob-5h-202309101303-28-00-00-03",
"imdb-rob-5h-202309101303-20-00-00-03",
"imdb-rob-5h-202309101303-09-00-00-03",
"imdb-rob-5h-202309101303-00-00-00-03",
"imdb-rob-5h-202309101303-90-00-00-02",
"imdb-rob-5h-202309101303-27-00-00-02",
"imdb-rob-5h-202309101303-18-00-00-02",
"imdb-rob-5h-202309101303-08-00-00-02",
"imdb-rob-5h-202309101303-00-00-00-02",
"imdb-rob-5h-202309101303-88-00-00-01",
"imdb-rob-5h-202309101303-29-00-00-01",
"imdb-rob-5h-202309101303-17-00-00-01",
"imdb-rob-5h-202309101303-09-00-00-01",
"imdb-rob-5h-202309101303-00-00-00-01"
]

In [10]:
training_jobs_roberta_5h_highlr = ["imdb-rob-5H-202309111917-88-00-00-05",
"imdb-rob-5H-202309111917-26-00-00-05",
"imdb-rob-5H-202309111917-18-00-00-05",
"imdb-rob-5H-202309111917-09-01-00-05",
"imdb-rob-5H-202309111917-00-00-00-05",
"imdb-rob-5H-202309111917-92-00-00-04",
"imdb-rob-5H-202309111917-28-00-00-04",
"imdb-rob-5H-202309111917-18-00-00-04",
"imdb-rob-5H-202309111917-09-00-00-04",
"imdb-rob-5H-202309111917-00-00-00-04",
"imdb-rob-5H-202309111917-87-00-00-03",
"imdb-rob-5H-202309111917-28-00-00-03",
"imdb-rob-5H-202309111917-20-00-00-03",
"imdb-rob-5H-202309111917-09-00-00-03",
"imdb-rob-5H-202309111917-00-00-00-03",
"imdb-rob-5H-202309111917-90-00-00-02",
"imdb-rob-5H-202309111917-27-00-00-02",
"imdb-rob-5H-202309111917-18-00-00-02",
"imdb-rob-5H-202309111917-08-00-00-02",
"imdb-rob-5H-202309111917-00-00-00-02",
"imdb-rob-5H-202309111917-88-00-00-01",
"imdb-rob-5H-202309111917-29-00-00-01",
"imdb-rob-5H-202309111917-17-00-00-01",
"imdb-rob-5H-202309111917-09-00-00-01",
"imdb-rob-5H-202309111917-00-00-00-01"]

In [11]:

training_jobs_roberta_imdb_2k_lowlr =["imdb-rob-2K-202309112045-89-00-00-05",
"imdb-rob-2K-202309112045-28-00-00-05",
"imdb-rob-2K-202309112045-18-00-00-05",
"imdb-rob-2K-202309112045-09-00-00-05",
"imdb-rob-2K-202309112045-00-00-00-05",
"imdb-rob-2K-202309112045-88-00-00-04",
"imdb-rob-2K-202309112045-28-00-00-04",
"imdb-rob-2K-202309112045-18-00-00-04",
"imdb-rob-2K-202309112045-09-00-00-04",
"imdb-rob-2K-202309112045-00-00-00-04",
"imdb-rob-2K-202309112045-89-00-00-03",
"imdb-rob-2K-202309112045-28-00-00-03",
"imdb-rob-2K-202309112045-18-00-00-03",
"imdb-rob-2K-202309112045-09-00-00-03",
"imdb-rob-2K-202309112045-00-00-00-03",
"imdb-rob-2K-202309112045-88-00-00-02",
"imdb-rob-2K-202309112045-28-00-00-02",
"imdb-rob-2K-202309112045-18-00-00-02",
"imdb-rob-2K-202309112045-09-00-00-02",
"imdb-rob-2K-202309112045-00-00-00-02",
"imdb-rob-2K-202309112045-88-00-00-01",
"imdb-rob-2K-202309112045-28-00-00-01",
"imdb-rob-2K-202309112045-19-00-00-01",
"imdb-rob-2K-202309112045-09-00-00-01",
"imdb-rob-2K-202309112045-00-00-00-01"]

In [12]:

s3_input_bert_vocab = "s3://aegovan-data/pretrained_models/bert-base-uncased/"
s3_input_roberta_vocab = "s3://aegovan-data/pretrained_models/roberta-base/"



In [13]:

# Add here
training_jobs = imdb_bert_2k_training_jobs 
s3_input_vocab = s3_input_bert_vocab
###

In [14]:
prediction_jobs = []





for dataset_name, dataset_uri in dataset.items():
    for training_job_name in training_jobs:

        item = {"job_name":f"{training_job_name}-{date_fmt}-{dataset_name}",
                    "model_uri" :f"s3://aegovan-data/sagemakerresults/{training_job_name}/output/model.tar.gz",
                    "s3_output_predictions": "s3://aegovan-data/counterfactual_{}/predictions_{}/{}".format(dataset_name,training_job_name,date_fmt),
                    "input_dataset" : dataset_uri,
                    "dataset_name" : dataset_name
                     }
        prediction_jobs.append(item)

In [15]:
prediction_jobs

[{'job_name': 'imdb202306041410-88-00-00-05-202309131559-contrastimpo',
  'model_uri': 's3://aegovan-data/sagemakerresults/imdb202306041410-88-00-00-05/output/model.tar.gz',
  's3_output_predictions': 's3://aegovan-data/counterfactual_contrastimpo/predictions_imdb202306041410-88-00-00-05/202309131559',
  'input_dataset': 's3://aegovan-data/contrast-set/test_imdb_contrast_plus_original.tsv',
  'dataset_name': 'contrastimpo'},
 {'job_name': 'imdb202306041410-26-00-00-05-202309131559-contrastimpo',
  'model_uri': 's3://aegovan-data/sagemakerresults/imdb202306041410-26-00-00-05/output/model.tar.gz',
  's3_output_predictions': 's3://aegovan-data/counterfactual_contrastimpo/predictions_imdb202306041410-26-00-00-05/202309131559',
  'input_dataset': 's3://aegovan-data/contrast-set/test_imdb_contrast_plus_original.tsv',
  'dataset_name': 'contrastimpo'},
 {'job_name': 'imdb202306041410-18-00-00-05-202309131559-contrastimpo',
  'model_uri': 's3://aegovan-data/sagemakerresults/imdb202306041410-18

### Run  prediction

In [16]:
#s3_output_predictions = "s3://aegovan-data/pubmed_asbtract/predictions_largescale_{}_{}/".format(job_prefix,date_fmt)
s3_data_type="S3Prefix"
usefilter=0
filter_threshold_std=1.0



In [17]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow
import random

processing_steps = []

PREPROCESSING_SCRIPT_LOCATION = "../src/inference/main_predict_sentiment_polarity.py"

s3_input_code = sagemaker_session.upload_data(
    PREPROCESSING_SCRIPT_LOCATION,
    bucket=sagemaker_session.default_bucket(),
    key_prefix="code/imdb/code",
)

for j in prediction_jobs:

    s3_model_path =  j["model_uri"]
    s3_output_predictions =j["s3_output_predictions"]
    eval_file =  j["input_dataset"]
    s3_input_data = eval_file
    dataset_name = j["dataset_name"]
    
    script_processor = ScriptProcessor(image_uri=docker_repo,
                                           command=["python"],
                                           env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                           role=role,
                                           instance_type=instance_type,
                                           instance_count=1,
                                           max_runtime_in_seconds= 5 * 24 * 60 * 60,
                                           volume_size_in_gb = 200,
                                           network_config=NetworkConfig(enable_network_isolation=False),
                                           base_job_name =f"{dataset_name}-inference"
                                           )


    sm_local_input_models = "/opt/ml/processing/input/data/models"
    sm_local_input_data = "/opt/ml/processing/input/data/jsondata"
    sm_local_input_vocab = "/opt/ml/processing/input/data/vocab"


    sm_local_output = "/opt/ml/processing/output"

    
    step_processing = sagemaker.ProcessingStep(f"imdb-{j['job_name']}", 
            script_processor, 
            j["job_name"], 
            container_entrypoint=['python', '/opt/ml/processing/input/code/main_predict_sentiment_polarity.py'],
                
            container_arguments=[
                f"{sm_local_input_data}/{eval_file.split('/')[-1]}",
                sm_local_input_models,
                sm_local_output,
                dataset_factory_map.get(dataset_name, dataset_name),
                "--tokenisor_data_dir", sm_local_input_vocab,           
                "--batch", "32"
            ],


            inputs=[
                ProcessingInput(
                                source=s3_input_code,
                                destination="/opt/ml/processing/input/code",
                                input_name="code",
                            ),
                ProcessingInput(
                        source=s3_input_data,
                        s3_data_type = s3_data_type,
                        destination=sm_local_input_data,
                        input_name="input-1",
                        s3_data_distribution_type="ShardedByS3Key"),

                ProcessingInput(
                        source=s3_model_path,
                        input_name="input-model",
                        destination=sm_local_input_models ,
                        s3_data_distribution_type="FullyReplicated"),

                ProcessingInput(
                        source=s3_input_vocab,
                        input_name="input-vocab",
                        destination=sm_local_input_vocab,
                        s3_data_distribution_type="FullyReplicated")
                
                
                ],


            outputs=[ProcessingOutput(
                    source=sm_local_output, 
                    destination=s3_output_predictions,
                    output_name='predictions')]
        )

    
    processing_steps.append(step_processing)



parallel_steps = []
max_parallel =10
#parallel_steps.append(states.Wait(f"wait", seconds=60*60*3))
for i in range(0, len(processing_steps), max_parallel):
    p = states.Parallel(f"predict-p-{i}")
    for si, s in enumerate(processing_steps[i: i+max_parallel]):
        w = (si+1)*30
        p.add_branch(Chain([states.Wait(f"wait-{i}-{w}", seconds=w) , s]))
    parallel_steps.append(p)

    
basic_path = Chain(parallel_steps)

basic_workflow = Workflow(
    name=f"imdb-infer-{date_fmt}", definition=basic_path, role=step_func_role
)

# Render the workflow
basic_workflow.render_graph()


In [18]:
basic_workflow.create()

'arn:aws:states:us-east-2:324346001917:stateMachine:imdb-infer-202309131559'

In [19]:
basic_workflow.execute()

Execution(execution_arn='arn:aws:states:us-east-2:324346001917:execution:imdb-infer-202309131559:3f1f440c-4611-43a2-b813-57fc57e674c2', name=None, status=<ExecutionStatus.Running: 'RUNNING'>, start_date=datetime.datetime(2023, 9, 13, 15, 59, 10, 470000, tzinfo=tzlocal()))